In [321]:
import dask.dataframe as dd
from dask_ml.model_selection import RandomizedSearchCV
import xgboost as xgb
from sklearn.metrics import accuracy_score
from joblib import dump,load
import numpy as np
import time

# load the classification model

In [324]:
import matplotlib.pyplot as plt
import pandas as pd

#load data and feature_important_axis1
datapath=r'Dataanalyse/axis2_demo_tablepart1_0.csv'
import json
with open('syntheic_data/feature_important_axis1.json') as f:
    feature_important_axis1 = json.load(f)

feature_important_axis1
df = pd.read_csv(datapath)
print(len(df))
df.drop_duplicates(subset=[df.columns[0]], inplace=True)
df["date"] = pd.to_datetime(df.Timestamp, unit='s')

features=["date",'Schadensklasse']+feature_important_axis1

print(features)
data_selected = df[features]
data_selected.head()

10000
['date', 'Schadensklasse', 'Effektivwert_z', 'Median_z', 'Mittlere_Absolute_Abweichung_z', 'Mittelwert_z', 'Median_y', 'Effektivwert_y', 'Effektivwert_x', 'Variance_z', 'Mittelwert_x', 'Standardabweichung_z']


,date,Schadensklasse,Effektivwert_z,Median_z,Mittlere_Absolute_Abweichung_z,Mittelwert_z,Median_y,Effektivwert_y,Effektivwert_x,Variance_z,Mittelwert_x,Standardabweichung_z
0,2020-11-04 11:40:00,2,15896.6,15872,350.131,15889.7,-512,571.115,657.526,220962,569.600,469.831
1,2020-11-04 11:40:00,2,15887.1,15872,363.095,15879.7,-512,571.746,627.592,235057,545.792,484.584
2,2020-11-04 11:40:00,2,15893.9,15872,395.262,15885.8,-512,566.507,651.770,256573,565.248,506.277
3,2020-11-04 11:40:00,2,15927.7,15872,389.710,15920.1,-512,568.931,653.578,241982,566.784,491.670
4,2020-11-04 11:40:00,2,15925.5,15872,414.196,15917.1,-512,586.402,661.006,268509,567.040,517.919


In [325]:
# define the length
alltime=np.unique(data_selected.date)
alltime
after=[]
for x in alltime:
    length=len(data_selected[data_selected["date"]==x])
    if length >=2000:
        after.append(x)
        print('length of batches: %d',length)
print('='*30)


length of batches: %d 2201
length of batches: %d 2348
length of batches: %d 2692


In [326]:
# only select the last batch 
for time in after:
    finaldata=data_selected[data_selected["date"]==time]

In [327]:
test=finaldata[feature_important_axis1].to_numpy()
test.shape

(2692, 10)

In [328]:
# load the axis1 model with 10 best features
model = load('10_best_xgboost_modeltestaxis1.joblib')
res=model.predict(test)
print(len(res))
res

2692


array([2, 2, 2, ..., 2, 2, 2], dtype=int64)

In [329]:
truedata=data_selected[data_selected["date"]==time]
true=truedata.Schadensklasse.to_list()
true=np.array(true)
print(len(true))
true

2692


array([2, 2, 2, ..., 2, 2, 2])

In [330]:
import numpy as np

# check the length of two arrays
if len(res) != len(true):
    raise ValueError("两个数组的长度不一致。")

# compare two arrays
similarity_array = res == true
print(similarity_array)
# calculate the number of similar elements
similar_count = np.sum(similarity_array)
print(similar_count)
# calculate the similarity ratio
similarity_ratio = similar_count / len(res)

print(f"相同元素的比例: {similarity_ratio:.2f}")


[ True  True  True ...  True  True  True]
2255
相同元素的比例: 0.84


# Set the config 

In [375]:

class Configs:
    output_attention = False
    enc_in = 10
    d_model = 512
    embed = 'timeF'
    freq = 's'
    dropout = 0.05
    e_layers = 2
    c_out = 10
    seq_len=200
    label_len=100
    pred_len=200
    step_size=pred_len
    model='Informer'
    dec_in=10
    factor=1
    n_heads=8
    d_ff=2048
    activation='gelu'
    distil=True
    d_layers=1
    batch_length=2000
configs = Configs()

# load the test data with length

In [376]:
# load the test dataset numpy array length 2000
import numpy as np  
import os
import pandas as pd
from joblib import dump,load
import torch
import torch.nn as nn
import pandas as pd
from datetime import timedelta
from Embed import DataEmbedding
from datetime import timedelta
from timefeatures import time_features
import torch
from torch.utils.data import Dataset, DataLoader,IterableDataset,ConcatDataset,TensorDataset
import pathlib as Path
import pickle
from models import Informer, Autoformer,LSTM,GRU



datapath=r'Dataanalyse/all_data_axis1.csv'
test_df = pd.read_csv(datapath)
test_df.drop_duplicates(subset=[test_df.columns[0]], inplace=True)
df = test_df.iloc[90000:200000].copy()
print(df.shape)
import json
with open('syntheic_data/feature_important_axis1.json') as f:
    feature_important_axis1 = json.load(f)
df["date"] = pd.to_datetime(df.Timestamp, unit='s')

features=["date",'Schadensklasse']+feature_important_axis1
data_selected = df[features]
print(data_selected.shape)
data=data_selected
alltime=np.unique(data.date)
after=[]
for x in alltime:
    length=len(data[data["date"]==x])
    damage=np.unique(data[data["date"]==x].Schadensklasse)
    if length >=configs.batch_length:
        after.append(x)
        print('length of batches: %d',length)
print('='*30)
# get the data
time=after[1]
print(time)

finaldata=data[data["date"]==time]
finaldata=finaldata[:configs.seq_len]
print(finaldata.shape)

(110000, 31)
(110000, 12)
length of batches: %d 8600
length of batches: %d 13834
length of batches: %d 17411
length of batches: %d 17419
length of batches: %d 17358
length of batches: %d 17363
length of batches: %d 16486
2021-05-25T10:53:20.000000000
(200, 12)


# Create the dataloader for predicting

In [377]:

# load the scaler
Scaler_file = r'D:\result\second\axis1_LSTM_Custom_axis1_ftM_sl200_ll100_pl200_dm512_nh8_el2_dl1_df2048_fc1_ebtimeF_dtTrue_test_0\data\scaler.pkl'
with open(Scaler_file, 'rb') as f:
    full_scaler = pickle.load(f)


#  Add the empty dataframe with double length for the original dataframe
zeros_df = pd.DataFrame(np.zeros((configs.pred_len, 12)), columns=finaldata.columns)
doubledata = pd.concat([finaldata, zeros_df], ignore_index=True)
print(doubledata.shape)  # shape of the output should be (400,10)
start_date = doubledata['date'].iloc[0]
# Generate a time series, starting from the start date, increasing every second, with the same length as data
time_series = [start_date + timedelta(seconds=i) for i in range(len(doubledata))]
# Assign the generated time series back to the 'date' column of data
doubledata.loc[:, 'date'] = time_series

dataset = doubledata
print('length of the testing dataset after times : %d', len(dataset))
df_stamp = dataset['date']
df_stamp.date = pd.to_datetime(df_stamp)

data_stamp = time_features(df_stamp, timeenc=1, freq='s')
data_stamp = torch.FloatTensor(data_stamp)
print(data_stamp.shape)

dataset = dataset.copy()

#dataset.drop(['date',"Schadensklasse"], axis=1, inplace=True)
dataset = pd.DataFrame(dataset, columns=finaldata.columns.drop(['date', 'Schadensklasse']))
dataset = full_scaler.transform(dataset)
dataset = torch.FloatTensor(dataset)


# Constructing samples
samples = []
Dte_samples=[]
count = 0
for index in range(0, len(dataset) - configs.seq_len - configs.pred_len + 1, configs.step_size):
    s_begin = index
    s_end = s_begin + configs.seq_len # 200/1000
    r_begin = s_end - configs.label_len # 100/500
    r_end = r_begin + configs.label_len + configs.pred_len #300/1500
    seq_x = dataset[s_begin:s_end] # 0-200/0-1000
    seq_y = dataset[r_begin:r_end] # 100-300/500-1500
    seq_x_mark = data_stamp[s_begin:s_end] # 0-200/0-1000
    seq_y_mark = data_stamp[r_begin:r_end] # 100-300/500-1500
    samples.append((seq_x, seq_y, seq_x_mark, seq_y_mark))
    count += 1


print('number of dataset in one batch: %d', count)
samples = DataLoader(dataset=samples, batch_size=32, shuffle=False, num_workers=0, drop_last=False, pin_memory=True)
print('='*30)
Dte_samples.append(samples)
print('number of batches: %d', len(samples))

class CustomDataset(Dataset):
    def __init__(self, dataloader):
        self.dataloader = dataloader
        self.dataset = dataloader.dataset
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        return self.dataset[idx]

Dte_datasets = [CustomDataset(dataloader) for dataloader in Dte_samples]
Dte_combined_dataset = ConcatDataset(Dte_datasets)
Dte_combined_dataloader = DataLoader(Dte_combined_dataset, batch_size=1, shuffle=False)

(400, 12)
length of the testing dataset after times : %d 400
torch.Size([400, 6])
number of dataset in one batch: %d 1
number of batches: %d 1


# load the time series model

In [378]:

def acquire_device():
    device = torch.device('cpu')
    print('Use CPU')
    return device

device = acquire_device()

def predict( batch_x, batch_y, batch_x_mark, batch_y_mark):
    # decoder input
    dec_inp = torch.zeros_like(batch_y[:, -configs.pred_len:, :]).float()
    dec_inp = torch.cat([batch_y[:, :configs.label_len, :], dec_inp], dim=1).float().to(device)
    # encoder - decoder

    def _run_model():
        outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
        return outputs
    outputs = _run_model()

    f_dim = 0
    outputs = outputs[:, -configs.pred_len:, f_dim:]
    batch_y = batch_y[:, -configs.pred_len:, f_dim:].to(device)

    return outputs, batch_y

Use CPU


In [379]:
model_dict = {
    'Autoformer': Autoformer,
    'Informer': Informer,
    'LSTM':LSTM,
    'GRU':GRU
}
model = model_dict[configs.model].Model(configs).to(device)

model_path = r"D:\result\second\axis1_Informer_Custom_axis1_ftM_sl200_ll100_pl200_dm512_nh8_el2_dl1_df2048_fc1_ebtimeF_dtTrue_test_0\checkpoints\checkpoint.pth"

model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

for i,(batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(Dte_combined_dataloader):
        batch_x = batch_x.float().to(device)
        batch_y = batch_y.float()

        batch_x_mark = batch_x_mark.float().to(device)
        batch_y_mark = batch_y_mark.float().to(device)
        #print('batch_x shape',batch_x.shape)

        outputs, batch_y = predict(batch_x, batch_y, batch_x_mark, batch_y_mark)
        
        print(outputs.shape)
        # Make sure the tensor is on the CPU
        outputs_cpu = outputs.cpu()
        # Convert to NumPy array
        outputs_numpy = outputs_cpu.detach().numpy()
        # Print the shape of the NumPy array to confirm the conversion
        print(outputs_numpy.shape)
        outputs_numpy=full_scaler.inverse_transform(outputs_numpy.reshape(-1,10))

        res=outputs_numpy#[1,:].reshape(1,10)
        print(res.shape)
        # load the axis1 model with 10 best features
        model = load('10_best_xgboost_modeltestaxis1.joblib')
        res=model.predict(res)
        print(res)
        print(len(res))


torch.Size([1, 200, 10])
(1, 200, 10)
(200, 10)
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
200


# Multiple batch

# set config

In [71]:
class Configs:
    output_attention = False
    enc_in = 10
    d_model = 512
    embed = 'timeF'
    freq = 's'
    dropout = 0.05
    e_layers = 2
    c_out = 10
    seq_len=200
    label_len=100
    pred_len=200
    step_size=pred_len
    model='Informer'
    dec_in=10
    factor=1
    n_heads=8
    d_ff=2048
    activation='gelu'
    distil=True
    d_layers=1
    batch_length=2000
configs = Configs()

# load all the data

In [85]:
# load the test dataset numpy array length 2000
import numpy as np  
import pandas as pd
from joblib import dump,load
import torch.nn as nn
import pandas as pd
from datetime import timedelta
from Embed import DataEmbedding
from torch.utils.data import Dataset, DataLoader, ConcatDataset
# load the test dataset numpy array length 2000
from timefeatures import time_features
import torch
from torch.utils.data import Dataset, DataLoader,IterableDataset,ConcatDataset,TensorDataset
import pathlib as Path
import pickle
from models import Informer, Autoformer,LSTM,GRU
from datetime import timedelta
from timefeatures import time_features
import torch
from torch.utils.data import Dataset, DataLoader,IterableDataset,ConcatDataset,TensorDataset

datapath=r'Dataanalyse/all_data_axis1.csv'
test_df = pd.read_csv(datapath)
test_df.drop_duplicates(subset=[test_df.columns[0]], inplace=True)
df = test_df.copy()
print(df.shape)
import json
with open('syntheic_data/feature_important_axis1.json') as f:
    feature_important_axis1 = json.load(f)
df["date"] = pd.to_datetime(df.Timestamp, unit='s')

features=["date",'Schadensklasse']+feature_important_axis1
data_selected = df[features]
print(data_selected.shape)
data=data_selected
alltime=np.unique(data.date)
after=[]
for x in alltime:
    length=len(data[data["date"]==x])
    damage=np.unique(data[data["date"]==x].Schadensklasse)
    if length >=configs.batch_length:
        after.append(x)
        print('length of batches: %d',length)
print('='*30)

(1771047, 31)
(1771047, 12)
length of batches: %d 14054
length of batches: %d 17530
length of batches: %d 17405
length of batches: %d 17397
length of batches: %d 17406
length of batches: %d 12516
length of batches: %d 13834
length of batches: %d 17411
length of batches: %d 17419
length of batches: %d 17358
length of batches: %d 17363
length of batches: %d 16533
length of batches: %d 13397
length of batches: %d 17330
length of batches: %d 17351
length of batches: %d 17319
length of batches: %d 17342
length of batches: %d 17419
length of batches: %d 17413
length of batches: %d 17395
length of batches: %d 10288
length of batches: %d 16685
length of batches: %d 17357
length of batches: %d 12804
length of batches: %d 3217
length of batches: %d 7583
length of batches: %d 7579
length of batches: %d 7582
length of batches: %d 7692
length of batches: %d 6647
length of batches: %d 3828
length of batches: %d 7591
length of batches: %d 7584
length of batches: %d 7588
length of batches: %d 7587
len

In [86]:
# load the scaler
Scaler_file = r'D:\result\second\axis1_LSTM_Custom_axis1_ftM_sl200_ll100_pl200_dm512_nh8_el2_dl1_df2048_fc1_ebtimeF_dtTrue_test_0\data\scaler.pkl'
with open(Scaler_file, 'rb') as f:
    full_scaler = pickle.load(f)


In [87]:
Dte_samples=[]
for time in after:
    finaldata=data[data["date"]==time]
    start_date = finaldata['date'].iloc[0]
    # Generate a time series, starting from the start date, increasing every second, with the same length as data
    time_series = [start_date + timedelta(seconds=i) for i in range(len(finaldata))]

    # Assign the generated time series back to the 'date' column of data
    finaldata.loc[:, 'date'] = time_series

    dataset = finaldata
    #print('length of the testing dataset after times 0.2: %d', len(dataset))

    df_stamp = dataset['date']
    df_stamp.date = pd.to_datetime(df_stamp)
    data_stamp = time_features(df_stamp, timeenc=1, freq='s')
    data_stamp = torch.FloatTensor(data_stamp)
    #print(data_stamp.shape)

    dataset = dataset.copy()
    # schadenklasse = dataset['Schadensklasse']
    dataset = pd.DataFrame(dataset, columns=finaldata.columns.drop(['date']))

    if 'Schadensklasse' in dataset.columns:
        withschaden = dataset.drop('Schadensklasse', axis=1)
    else:
        withschaden = dataset.copy()

    trans_dataset = full_scaler.transform(withschaden)
    trans_dataset = torch.FloatTensor(trans_dataset)

    samples = []
    count = 0
    #print('range',range(0, len(dataset) - configs.seq_len - configs.pred_len + 1))
    #print('step size',configs.step_size)
    for index in range(0, len(dataset) - configs.seq_len - configs.pred_len + 1, configs.step_size):
        # train_x, x_mark, train_y, y_mark
        s_begin = index
        s_end = s_begin + configs.seq_len
        r_begin = s_end - configs.label_len
        r_end = r_begin + configs.label_len + configs.pred_len
        damage=dataset['Schadensklasse'][r_begin:r_end].to_numpy()
        damage = torch.LongTensor(damage)

        #print("damage",damage[10])
        old_seq_x = dataset[s_begin:s_end].copy()
        old_seq_y = dataset[r_begin:r_end].copy()

        if 'Schadensklasse' in old_seq_x.columns:
            seq_x = old_seq_x.drop('Schadensklasse', axis=1)
        else:
            seq_x = old_seq_x.copy()
            
        if 'Schadensklasse' in old_seq_y.columns:
            seq_y = old_seq_y.drop('Schadensklasse', axis=1)
        else:
            seq_y = old_seq_y.copy()      


        seq_x = full_scaler.transform(seq_x)
        seq_y = full_scaler.transform(seq_y)

        seq_x = torch.FloatTensor(seq_x)
        seq_y = torch.FloatTensor(seq_y)

        seq_x_mark = data_stamp[s_begin:s_end]
        seq_y_mark = data_stamp[r_begin:r_end]

        samples.append((seq_x, seq_y, seq_x_mark, seq_y_mark,damage))
        count += 1


    #print('number of dataset in one batch: %d', count)
    samplesloader = DataLoader(dataset=samples, batch_size=32, shuffle=False, num_workers=0, drop_last=False, pin_memory=True)
    #print('length of samples',len(samplesloader))
    for x,y, z, p ,damge in samplesloader:
       pass  
    #print('the last shape',damge.shape)
    
    #print('='*30)
    Dte_samples.append(samplesloader)
print('number of batches: %d', len(Dte_samples))
    

number of batches: %d 179


In [88]:
trans_dataset.shape

torch.Size([8747, 10])

In [89]:
class CustomDataset(Dataset):
    def __init__(self, dataloader):
        self.dataloader = dataloader
        self.dataset = dataloader.dataset
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        return self.dataset[idx]


Dte_datasets = [CustomDataset(dataloader) for dataloader in Dte_samples]
Dte_combined_dataset = ConcatDataset(Dte_datasets)
Dte_combined_dataloader = DataLoader(Dte_combined_dataset, batch_size=1, shuffle=False)

print(len(Dte_combined_dataloader))

8454


In [77]:
for i, (x,y,z,p,da ) in enumerate(Dte_combined_dataloader) :
    print(da.shape)
    pass
print(i)

torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1, 300])
torch.Size([1

# load the time series model

In [90]:

def acquire_device():
    device = torch.device('cuda')
    print('Use gpu')
    return device

device = acquire_device()



Use gpu


In [92]:
import torch
import pathlib as Path
import pickle
from models import Informer, Autoformer,LSTM,GRU
from datetime import timedelta
from timefeatures import time_features
from torch.utils.data import Dataset, DataLoader,IterableDataset,ConcatDataset,TensorDataset

model_dict = {
    'Autoformer': Autoformer,
    'Informer': Informer,
    'LSTM':LSTM,
    'GRU':GRU
}
timeserie_model = model_dict[configs.model].Model(configs).to(device)

model_path = r"D:\result\second\axis1_Informer_Custom_axis1_ftM_sl200_ll100_pl200_dm512_nh8_el2_dl1_df2048_fc1_ebtimeF_dtTrue_test_0\checkpoints\checkpoint.pth"

timeserie_model.load_state_dict(torch.load(model_path, map_location=device))
timeserie_model.eval()


def acquire_device():
    device = torch.device('cpu')
    print('Use gpu')
    return device

device = acquire_device()

def predict( batch_x, batch_y, batch_x_mark, batch_y_mark):
    # decoder input
    dec_inp = torch.zeros_like(batch_y[:, -configs.pred_len:, :]).float()
    dec_inp = torch.cat([batch_y[:, :configs.label_len, :], dec_inp], dim=1).float().to(device)
    # encoder - decoder

    def run_model():
        outputs = timeserie_model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
        return outputs
    outputs = run_model()

    f_dim = 0
    outputs = outputs[:, -configs.pred_len:, f_dim:]
    batch_y = batch_y[:, -configs.pred_len:, f_dim:].to(device)

    return outputs, batch_y


pre=[]
groundtrue=[]
for i,(batch_x, batch_y, batch_x_mark, batch_y_mark,truedamage) in enumerate(Dte_combined_dataloader):
        batch_x = batch_x.float().to(device)
        batch_y = batch_y.float()#.to(device)

        batch_x_mark = batch_x_mark.float().to(device)
        batch_y_mark = batch_y_mark.float().to(device)
        # print('batch_x shape',batch_x.shape)
        #truedamage=truedamage.float().to(device)
        outputs, batch_y = predict(batch_x, batch_y, batch_x_mark, batch_y_mark)
        
        # Convert to NumPy array
        outputs_numpy = outputs.detach().numpy()
        # Print the shape of the NumPy array to confirm the conversion
        #print(outputs_numpy.shape)
        outputs_numpy=full_scaler.inverse_transform(outputs_numpy.reshape(-1,10))

        res=outputs_numpy#[1,:].reshape(1,10)


        # load the axis1 model with 10 best features
        model = load('10_best_xgboost_modeltestaxis1.joblib')
        predresult=model.predict(res)
        #print("res",predresult.shape)
        pre.append(predresult)

        true=truedamage[:,-configs.pred_len:]
        output_true=true.detach().numpy().reshape(-1)
        #print("true",output_true.shape)
        groundtrue.append(output_true)
        #print("*"*30)


Use gpu


In [93]:
print(len(pre))
print(pre[0])
print(len(groundtrue))
print(groundtrue[0])

8454
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
8454
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [94]:
# 使用列表推导式将二维列表转换为一维列表
pre_flattened = [item for sublist in pre for item in sublist]
groundtrue_flattened = [item for sublist in groundtrue for item in sublist]

# 打印结果以确认
print(len(pre_flattened))  # 应该输出 8400 (42 * 200)
print(len(groundtrue_flattened))  # 应该输出 8400 (42 * 200)

# 确认顺序不变
print(pre_flattened[:200])  # 打印前10个元素确认顺序
print(groundtrue_flattened[:200])  

1690800
1690800
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2

In [95]:
print(type(pre_flattened))
print(type(groundtrue_flattened))

<class 'list'>
<class 'list'>


In [96]:
import numpy as np


pre_array = np.array(pre_flattened)
groundtrue_array = np.array(groundtrue_flattened)

# 检查两个数组的长度是否一致
if len(pre_array) != len(groundtrue_array):
    raise ValueError("两个数组的长度不一致。")

# 比较两个数组
similarity_array = pre_array == groundtrue_array

# 计算相同元素的数量
similar_count = np.sum(similarity_array)

# 计算相似度比例
similarity_ratio = similar_count / len(pre_array)

print(f"相同元素的比例: {similarity_ratio:.2f}")

相同元素的比例: 0.87


In [467]:
flattened_pre = [item for sublist in pre for item in sublist]

print(flattened_pre)

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [403]:
import numpy as np
np.unique(flattened_pre)

array([1, 2], dtype=int64)

In [93]:


# load the scaler
import pathlib as Path
import pickle
Scaler_file = r'D:\result\second\axis1_LSTM_Custom_axis1_ftM_sl200_ll100_pl200_dm512_nh8_el2_dl1_df2048_fc1_ebtimeF_dtTrue_test_0\data\scaler.pkl'
with open(Scaler_file, 'rb') as f:
    full_scaler = pickle.load(f)


Dte_samples=[]
finaldata=data[data["date"]==time]
start_date = finaldata['date'].iloc[0]
# 生成一个时间序列，从起始日期开始，每秒递增，长度与data相同
time_series = [start_date + timedelta(seconds=i) for i in range(len(finaldata))]

# 将生成的时间序列赋值回data的'date'列
finaldata.loc[:, 'date'] = time_series

dataset = finaldata
print('length of the testing dataset after times 0.2: %d', len(dataset))

df_stamp = dataset['date']
df_stamp.date = pd.to_datetime(df_stamp)
data_stamp = time_features(df_stamp, timeenc=1, freq='s')
data_stamp = torch.FloatTensor(data_stamp)
print(data_stamp.shape)

dataset = dataset.copy()

#dataset.drop(['date',"Schadensklasse"], axis=1, inplace=True)
dataset = pd.DataFrame(dataset, columns=finaldata.columns.drop(['date', 'Schadensklasse']))

dataset = full_scaler.transform(dataset)
dataset = torch.FloatTensor(dataset)
# 构造样本
samples = []
count = 0
for index in range(0, len(dataset) - configs.seq_len - configs.pred_len + 1, configs.step_size):
    # train_x, x_mark, train_y, y_mark
    s_begin = index
    s_end = s_begin + configs.seq_len
    r_begin = s_end - configs.label_len
    r_end = r_begin + configs.label_len + configs.pred_len
    seq_x = dataset[s_begin:s_end]
    seq_y = dataset[r_begin:r_end]
    seq_x_mark = data_stamp[s_begin:s_end]
    seq_y_mark = data_stamp[r_begin:r_end]
    samples.append((seq_x, seq_y, seq_x_mark, seq_y_mark))
    count += 1


print('number of dataset in one batch: %d', count)
samples = DataLoader(dataset=samples, batch_size=32, shuffle=False, num_workers=0, drop_last=False, pin_memory=True)
print('='*30)
Dte_samples.append(samples)
print('number of batches: %d', len(samples))


length of the testing dataset after times 0.2: %d 8600
torch.Size([8600, 6])
number of dataset in one batch: %d 42
number of batches: %d 2


In [90]:
for x,y,z,p in samples:
    print(x.shape)
    print(y.shape)
    print(z.shape)
    print(p.shape)
    break


torch.Size([32, 200, 10])
torch.Size([32, 300, 10])
torch.Size([32, 200, 6])
torch.Size([32, 300, 6])


In [ ]:
# from datetime import timedelta
# from timefeatures import time_features
# import torch
# from torch.utils.data import Dataset, DataLoader,IterableDataset,ConcatDataset,TensorDataset

# # load the scaler
# import pathlib as Path
# import pickle
# Scaler_file = r'D:\result\second\axis1_LSTM_Custom_axis1_ftM_sl200_ll100_pl200_dm512_nh8_el2_dl1_df2048_fc1_ebtimeF_dtTrue_test_0\data\scaler.pkl'
# with open(Scaler_file, 'rb') as f:
#     full_scaler = pickle.load(f)


# Dte_samples=[]
# for time in after:
#     finaldata=data[data["date"]==time]
#     start_date = finaldata['date'].iloc[0]
#     # 生成一个时间序列，从起始日期开始，每秒递增，长度与data相同
#     time_series = [start_date + timedelta(seconds=i) for i in range(len(finaldata))]

#     # 将生成的时间序列赋值回data的'date'列
#     finaldata.loc[:, 'date'] = time_series

#     dataset = finaldata
#     print('length of the testing dataset after times 0.2: %d', len(dataset))

#     df_stamp = dataset['date']
#     df_stamp.date = pd.to_datetime(df_stamp)
#     data_stamp = time_features(df_stamp, timeenc=1, freq='s')
#     data_stamp = torch.FloatTensor(data_stamp)
#     print(data_stamp.shape)

#     dataset = dataset.copy()

#     dataset.drop(['date',"Schadensklasse"], axis=1, inplace=True)
#     dataset = pd.DataFrame(dataset, columns=finaldata.columns.drop(['date', 'Schadensklasse']))

#     dataset = full_scaler.transform(dataset)
#     dataset = torch.FloatTensor(dataset)
#     # 构造样本
#     samples = []
#     count = 0
#     for index in range(0, len(dataset) - configs.seq_len - configs.pred_len + 1, configs.step_size):
#         # train_x, x_mark, train_y, y_mark
#         s_begin = index
#         s_end = s_begin + configs.seq_len
#         r_begin = s_end - configs.label_len
#         r_end = r_begin + configs.label_len + configs.pred_len
#         seq_x = dataset[s_begin:s_end]
#         seq_y = dataset[r_begin:r_end]
#         seq_x_mark = data_stamp[s_begin:s_end]
#         seq_y_mark = data_stamp[r_begin:r_end]
#         samples.append((seq_x, seq_y, seq_x_mark, seq_y_mark))
#         count += 1


#     print('number of dataset in one batch: %d', count)
#     samples = DataLoader(dataset=samples, batch_size=32, shuffle=False, num_workers=0, drop_last=False, pin_memory=True)
#     print('='*30)
#     Dte_samples.append(samples)
# print('number of batches: %d', len(Dte_samples))

# Single batch

In [101]:
for i,(batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(Dte_combined_dataloader):
    print(batch_x.shape)
    print(i)

torch.Size([1, 200, 10])
0
torch.Size([1, 200, 10])
1
torch.Size([1, 200, 10])
2
torch.Size([1, 200, 10])
3
torch.Size([1, 200, 10])
4
torch.Size([1, 200, 10])
5
torch.Size([1, 200, 10])
6
torch.Size([1, 200, 10])
7
torch.Size([1, 200, 10])
8
torch.Size([1, 200, 10])
9
torch.Size([1, 200, 10])
10
torch.Size([1, 200, 10])
11
torch.Size([1, 200, 10])
12
torch.Size([1, 200, 10])
13
torch.Size([1, 200, 10])
14
torch.Size([1, 200, 10])
15
torch.Size([1, 200, 10])
16
torch.Size([1, 200, 10])
17
torch.Size([1, 200, 10])
18
torch.Size([1, 200, 10])
19
torch.Size([1, 200, 10])
20
torch.Size([1, 200, 10])
21
torch.Size([1, 200, 10])
22
torch.Size([1, 200, 10])
23
torch.Size([1, 200, 10])
24
torch.Size([1, 200, 10])
25
torch.Size([1, 200, 10])
26
torch.Size([1, 200, 10])
27
torch.Size([1, 200, 10])
28
torch.Size([1, 200, 10])
29
torch.Size([1, 200, 10])
30
torch.Size([1, 200, 10])
31
torch.Size([1, 200, 10])
32
torch.Size([1, 200, 10])
33
torch.Size([1, 200, 10])
34
torch.Size([1, 200, 10])
35
to

In [102]:
import torch
import torch.nn as nn
import pandas as pd
from datetime import timedelta
from Embed import DataEmbedding
from torch.utils.data import Dataset, DataLoader, ConcatDataset


# Define your model architecture
class Model(nn.Module):
    def __init__(self, configs):
        super(Model, self).__init__()
        self.pred_len = configs.pred_len
        self.output_attention = configs.output_attention

        # Embedding layer
        self.enc_embedding = DataEmbedding(configs.enc_in, configs.d_model, configs.embed, configs.freq, configs.dropout)
        
        # LSTM Layer
        self.lstm = nn.LSTM(
            input_size=configs.d_model,
            hidden_size=configs.d_model,
            num_layers=configs.e_layers,
            bidirectional=False,
            batch_first=True
        )
        
        # Projection layer
        num_directions = 1
        self.projection = nn.Linear(configs.d_model * num_directions, configs.c_out)

    def forward(self, x_enc, x_mark_enc, x_dec, x_mark_dec, enc_self_mask=None):
        x_enc = self.enc_embedding(x_enc, x_mark_enc)
        lstm_out, _ = self.lstm(x_enc)
        lstm_out = self.projection(lstm_out)

        if self.output_attention:
            return lstm_out[:, -self.pred_len:, :], None
        else:
            return lstm_out[:, -self.pred_len:, :]

In [103]:
import os
import torch
def acquire_device():
    device = torch.device('cpu')
    print('Use CPU')
    return device
device = acquire_device()
import torch

model_path = r"D:\result\second\axis2_LSTM_Custom_axis2_ftM_sl200_ll100_pl200_dm512_nh8_el2_dl1_df2048_fc1_ebtimeF_dtTrue_test_0\checkpoints\checkpoint.pth"
model = Model(configs).to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()


def predict( batch_x, batch_y, batch_x_mark, batch_y_mark):
    # decoder input
    dec_inp = torch.zeros_like(batch_y[:, -200:, :]).float()
    dec_inp = torch.cat([batch_y[:, :100, :], dec_inp], dim=1).float().to(device)
    # encoder - decoder

    def _run_model():
        outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
        return outputs
    outputs = _run_model()

    f_dim = 0
    outputs = outputs[:, -200:, f_dim:]
    batch_y = batch_y[:, -200:, f_dim:].to(device)

    return outputs, batch_y

Use CPU


In [104]:
for i,(batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(Dte_combined_dataloader):
        batch_x = batch_x.float().to(device)
        batch_y = batch_y.float()

        batch_x_mark = batch_x_mark.float().to(device)
        batch_y_mark = batch_y_mark.float().to(device)
        #print('batch_x shape',batch_x.shape)

        outputs, batch_y = predict(batch_x, batch_y, batch_x_mark, batch_y_mark)
        print(outputs.shape)

torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])


In [108]:
outputs[0].shape

torch.Size([200, 10])

In [123]:
# 确保张量在 CPU 上
outputs_cpu = outputs[0].cpu()
# 转换为 NumPy 数组
outputs_numpy = outputs_cpu.detach().numpy()
# 打印 NumPy 数组的形状以确认转换
print(outputs_numpy.shape)
outputs_numpy=full_scaler.inverse_transform(outputs_numpy)

res=outputs_numpy[1,:].reshape(1,10)
res.shape
# load the axis1 model with 10 best features
model = load('10_best_xgboost_modeltestaxis1.joblib')
res=model.predict(res)
res


(200, 10)


In [139]:
outputs_numpy.shape

(200, 10)

In [141]:
res=outputs_numpy[1,:].reshape(1,10)
res.shape

(1, 10)

In [142]:
# load the axis1 model with 10 best features
model = load('10_best_xgboost_modeltestaxis1.joblib')
res=model.predict(res)
res

array([2], dtype=int64)